In [ ]:
! pip install datasets transformers

In [ ]:
from huggingface_hub import notebook_login

notebook_login()

In [ ]:
from datasets import load_dataset

dataset = load_dataset("Vishaltiwari2019/textGen-databricks-dolly",split = 'train')

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


Generating train split: 0 examples [00:00, ? examples/s]

In [ ]:
dataset.train_test_split(test_size=0.2)

DatasetDict({
    train: Dataset({
        features: ['text'],
        num_rows: 8435
    })
    test: Dataset({
        features: ['text'],
        num_rows: 2109
    })
})

In [ ]:
model_checkpoint = "distilgpt2"

In [ ]:
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained(model_checkpoint, use_fast=True)

tokenizer_config.json:   0%|          | 0.00/26.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/762 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/1.04M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

In [ ]:
def tokenize_function(examples):
    return tokenizer(examples["text"])

In [ ]:
tokenized_datasets = dataset.map(tokenize_function, batched=True, num_proc=4, remove_columns=["text"])

/usr/local/lib/python3.10/dist-packages/multiprocess/popen_fork.py:66: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
  self.pid = os.fork()


Map (num_proc=4):   0%|          | 0/10544 [00:00<?, ? examples/s]

Token indices sequence length is longer than the specified maximum sequence length for this model (1555 > 1024). Running this sequence through the model will result in indexing errors
/usr/local/lib/python3.10/dist-packages/multiprocess/popen_fork.py:66: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
  self.pid = os.fork()
Token indices sequence length is longer than the specified maximum sequence length for this model (1042 > 1024). Running this sequence through the model will result in indexing errors
Token indices sequence length is longer than the specified maximum sequence length for this model (1937 > 1024). Running this sequence through the model will result in indexing errors
Token indices sequence length is longer than the specified maximum sequence length for this model (1144 > 1024). Running this sequence through the model will result in indexing errors


In [ ]:
tokenized_datasets

Dataset({
    features: ['input_ids', 'attention_mask'],
    num_rows: 10544
})

In [ ]:
# block_size = tokenizer.model_max_length
block_size = 128

In [ ]:
def group_texts(examples):
    # Concatenate all texts.
    concatenated_examples = {k: sum(examples[k], []) for k in examples.keys()}
    total_length = len(concatenated_examples[list(examples.keys())[0]])
    # We drop the small remainder, we could add padding if the model supported it instead of this drop, you can
        # customize this part to your needs.
    total_length = (total_length // block_size) * block_size
    # Split by chunks of max_len.
    result = {
        k: [t[i : i + block_size] for i in range(0, total_length, block_size)]
        for k, t in concatenated_examples.items()
    }
    result["labels"] = result["input_ids"].copy()
    return result

In [ ]:
lm_datasets = tokenized_datasets.map(
    group_texts,
    batched=True,
    batch_size=1000,
    num_proc=4,
)

Map (num_proc=4):   0%|          | 0/10544 [00:00<?, ? examples/s]

In [ ]:
lm_datasets

Dataset({
    features: ['input_ids', 'attention_mask', 'labels'],
    num_rows: 9039
})

In [ ]:
tokenizer.decode(lm_datasets["input_ids"][1])

'John D RockerfellerInstruction:\nWhy mobile is bad for human\n\nResponse:\nWe are always engaged one phone which is not good.Instruction:\nWhat is a polygon?\n\nResponse:\nA polygon is a form in Geometry.  It is a single dimensional plane made of connecting lines and any number of vertices.  It is a closed chain of connected line segments or edges.  The vertices of the polygon are formed where two edges meet.  Examples of polygons are hexagons, pentagons, and octagons.  Any plane that does not contain edges or'

In [ ]:
from transformers import AutoModelForCausalLM
model = AutoModelForCausalLM.from_pretrained(model_checkpoint)

In [ ]:
from transformers import Trainer, TrainingArguments

In [ ]:
model_name = model_checkpoint.split("/")[-1]
training_args = TrainingArguments(
    f"{model_name}-finetuned-databricks",
    evaluation_strategy = "epoch",
    learning_rate=2e-5,
    weight_decay=0.01,
    num_train_epochs=0.6,
    push_to_hub=True,
)

In [ ]:
lm_datasets = lm_datasets.train_test_split(test_size = 0.2)

In [ ]:
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=lm_datasets["train"],
    eval_dataset=lm_datasets["test"],
)

/usr/local/lib/python3.10/dist-packages/accelerate/accelerator.py:436: FutureWarning: Passing the following arguments to `Accelerator` is deprecated and will be removed in version 1.0 of Accelerate: dict_keys(['dispatch_batches', 'split_batches', 'even_batches', 'use_seedable_sampler']). Please pass an `accelerate.DataLoaderConfiguration` instead: 
dataloader_config = DataLoaderConfiguration(dispatch_batches=None, split_batches=False, even_batches=True, use_seedable_sampler=True)
  warnings.warn(


In [ ]:
trainer.train()

Epoch,Training Loss,Validation Loss


Epoch,Training Loss,Validation Loss
0,3.440400,3.237592


TrainOutput(global_step=543, training_loss=3.4370443438956753, metrics={'train_runtime': 6573.5567, 'train_samples_per_second': 0.66, 'train_steps_per_second': 0.083, 'total_flos': 141884135571456.0, 'train_loss': 3.4370443438956753, 'epoch': 0.6})

In [ ]:
trainer.push_to_hub()

model.safetensors:   0%|          | 0.00/328M [00:00<?, ?B/s]

Upload 2 LFS files:   0%|          | 0/2 [00:00<?, ?it/s]

events.out.tfevents.1712818520.67005e1eac97.4526.2:   0%|          | 0.00/5.81k [00:00<?, ?B/s]

CommitInfo(commit_url='https://huggingface.co/Vishaltiwari2019/distilgpt2-finetuned-databricks/commit/028e344d87f22ef4bef06f7ea43d4f3ca7e4d4ba', commit_message='End of training', commit_description='', oid='028e344d87f22ef4bef06f7ea43d4f3ca7e4d4ba', pr_url=None, pr_revision=None, pr_num=None)

CommitInfo(commit_url='https://huggingface.co/Vishaltiwari2019/distilgpt2-finetuned-databricks/commit/6e294cc51664317e992c938c9b0aec931665543b', commit_message='End of training', commit_description='', oid='6e294cc51664317e992c938c9b0aec931665543b', pr_url=None, pr_revision=None, pr_num=None)

In [ ]:
!pip install transformers[torch]===4.39.3